In [4]:
import pandas as pd

# 경로
ver9_path = "/content/drive/MyDrive/cohort_ver9_fill_icustays.csv"
patients_path = "/content/drive/MyDrive/patients.csv"
out_path = "/content/drive/MyDrive/cohort_ver10_fill_patients.csv"

# 로드
ver9 = pd.read_csv(ver9_path)
patients = pd.read_csv(patients_path)

# 중복 컬럼 제거
ver9 = ver9.loc[:, ~ver9.columns.duplicated()]
patients = patients.loc[:, ~patients.columns.duplicated()]

# 키 존재 확인
if "subject_id" not in ver9.columns:
    raise ValueError("ver9에 subject_id가 없습니다.")

# 타입 정규화
ver9["subject_id"] = ver9["subject_id"].astype(str)

# patients 컬럼명 정규화(공백 제거, 소문자)
patients.columns = [c.strip() for c in patients.columns]
patients.rename(columns={c: c.lower() for c in patients.columns}, inplace=True)
if "subject_id" not in patients.columns:
    raise ValueError("patients에 subject_id가 없습니다.")
patients["subject_id"] = patients["subject_id"].astype(str)

# patients 원본 후보
pat_candidates = ["subject_id", "gender", "anchor_age", "anchor_year", "anchor_year_group", "dod"]

# ver9에도 존재하는 컬럼만 fill 대상으로 선택(subject_id 제외)
fillable_cols = sorted(set(ver9.columns).intersection(pat_candidates) - {"subject_id"})

# 보조 신호로 쓸 dod는 ver9에 없어도 반드시 끌고 온다
aux_cols = []
if "dod" in patients.columns:
    aux_cols.append("dod")

# subject_id 중복 처리(최신 anchor_year 우선)
patients_tmp = patients.copy()
if "anchor_year" in patients_tmp.columns:
    patients_tmp = patients_tmp.sort_values(["subject_id", "anchor_year"])
    patients_tmp = patients_tmp.drop_duplicates(subset=["subject_id"], keep="last")
else:
    patients_tmp = patients_tmp.drop_duplicates(subset=["subject_id"], keep="last")

# 병합에 사용할 컬럼
keep_cols = ["subject_id"] + fillable_cols + aux_cols
keep_cols = [c for c in keep_cols if c in patients_tmp.columns]
patients_tmp = patients_tmp[keep_cols]

# 병합
merged = ver9.merge(patients_tmp, on="subject_id", how="left", suffixes=("", "_pat"))

def is_blank(s):
    if s.dtype == object:
        return s.isna() | (s.astype(str).str.strip() == "")
    return s.isna()

filled_counts = {}

# ver9의 결측을 patients 값으로 채움
for col in fillable_cols:
    pat_col = f"{col}_pat"
    if pat_col in merged.columns:
        mask_to_fill = is_blank(merged[col]) & (~merged[pat_col].isna())
        filled_counts[col] = int(mask_to_fill.sum())
        merged.loc[mask_to_fill, col] = merged.loc[mask_to_fill, pat_col]
        merged.drop(columns=[pat_col], inplace=True, errors="ignore")

# death_flag 보완: dod가 있으면 1로 설정
if "death_flag" in merged.columns:
    # 우선 숫자/문자 혼합 방지
    # death_flag가 전혀 없으면 새로 만들고 0으로 채움
    if merged["death_flag"].dtype == object:
        # object이지만 숫자 문자열일 수 있음
        merged["death_flag"] = pd.to_numeric(merged["death_flag"], errors="coerce")
    merged["death_flag"] = merged["death_flag"].fillna(0)

    # 1) patients.dod 기반
    if "dod" in merged.columns:
        mask_dod = ~merged["dod"].isna()
        merged.loc[mask_dod, "death_flag"] = 1
        filled_counts["death_flag_from_dod"] = int(mask_dod.sum())

    # 2) admissions.hospital_expire_flag 기반
    if "hospital_expire_flag" in merged.columns:
        # hospital_expire_flag가 1인 경우
        mask_hef = (merged["hospital_expire_flag"] == 1) & (merged["death_flag"] != 1)
        merged.loc[mask_hef, "death_flag"] = 1
        filled_counts["death_flag_from_hospital_expire_flag"] = int(mask_hef.sum())

    # 3) admissions.deathtime 기반
    if "deathtime" in merged.columns:
        mask_deathtime = ~merged["deathtime"].isna()
        merged.loc[mask_deathtime, "death_flag"] = 1
        filled_counts["death_flag_from_deathtime"] = int(mask_deathtime.sum())

    # 최종 0/1 정규화
    merged["death_flag"] = merged["death_flag"].fillna(0).astype(int)

# 저장 전, 보조 dod 컬럼은 유지할지 여부 결정(여기서는 안전 위해 남김)
# 필요 없으면 아래 줄 주석 해제
# if "dod" in merged.columns:
#     merged.drop(columns=["dod"], inplace=True)

merged.to_csv(out_path, index=False)

print("Saved:", out_path)
print("ver9 rows:", len(ver9))
print("patients rows:", len(patients))
print("merged rows:", len(merged))
print("fillable columns:", fillable_cols)
print("filled cell counts per column:", filled_counts)


Saved: /content/drive/MyDrive/cohort_ver10_fill_patients.csv
ver9 rows: 836
patients rows: 364627
merged rows: 836
fillable columns: ['gender']
filled cell counts per column: {'gender': 0, 'death_flag_from_dod': 297}
